In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from scipy.spatial.transform import Rotation
import glob
import os
import json

In [2]:
p_space = re.compile('( *)(.*)')

def remove_space(x):
    name = p_space.match(x)
    return name.group(2)

# extracting transformation parameters (rotation angles and translation vector) from csv
def extract_transformation(x):
    # converting to rotations
    r = Rotation.from_euler('XYZ', [x['pose_Rx'], x['pose_Ry'], x['pose_Rz']], degrees=False)
    # converting to translation vector
    translation = np.array([x['pose_Tx'], x['pose_Ty'], x['pose_Tz']])
    return r, translation


# mapping global coordinate to local ones using the inverse transformation defined by the Pose parameters
def map_row_to_local_quat(x):
    r, translation = extract_transformation(x)
    
    # Each row is a (possibly non-unit norm) quaternion in scalar-last (x, y, z, w) format.
    quat = r.as_quat()
    x['quat_x'] = quat[0]  # x
    x['quat_y'] = quat[1]  # y
    x['quat_z'] = quat[2]  # z
    x['quat_w'] = quat[3]  # w
    
    # mapping each of the 68 landmarks points to local coordinates
    for i in range(0, 68):
        # converting to location vector
        vec_global = np.array([x['X_' + str(i)], x['Y_' + str(i)], x['Z_' + str(i)]])
        # translating to object space and inverting rotation
        vec_local = r.apply((vec_global - translation), inverse=True)
        # updating row with local coordinates
        x['X_L_' + str(i)] = vec_local[0]
        x['Y_L_' + str(i)] = vec_local[1]
        x['Z_L_' + str(i)] = vec_local[2]

    return x

In [7]:
# please add an absolute path to the processed directory of OpenFace
video_path = "C:\\Users\\morda\\Downloads\\OpenFace_2.2.0_win_x64\\to_processed\\"
#input_paths = [".\\samples\\test\\openface_landmarks.csv"]
paths = glob.glob(video_path + "*.csv")

# please specify a location where to save the processed CSVs
save_location = "C:\\Users\\morda\\Downloads\\OpenFace_2.2.0_win_x64\\after_processed\\" 

In [8]:
# preprocessing
for path in tqdm(paths):
    df = pd.read_csv(path)
    df = df.rename(columns=remove_space)
    df = df.apply(map_row_to_local_quat, axis=1)
    df = df.drop(['face_id', 'success', 'timestamp', 'frame','confidence',
                  'pose_Tz', 'pose_Ty', 'pose_Tx', 'pose_Rx', 'pose_Ry',
                  'pose_Rz'], axis=1)
    for i in range(0, 68):
        df = df.drop(["X_" + str(i), "Y_" + str(i), "Z_" + str(i)], axis=1)
    
    #df.to_csv(save_location)

    base_name = os.path.basename(path)
    df.to_csv(str(save_location) + "\\" + str(base_name) + "_proc" + ".csv")
    df.to_json(str(save_location) + "\\" + str(base_name) + "_proc" + ".json", orient="index")

In [ ]:
# an example of how the CSV looks
df.head()

In [ ]:
# an exmaple of how the json looks like
with open(str(save_location) + "\\" + str(base_name) + "_proc" + ".json", encoding='utf-8') as data_file:
    data = json.loads(data_file.read())
data

In [ ]:
for i in range(0, 68):
    print (["X_" + str(i), "Y_" + str(i), "Z_" + str(i)])